In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import boto3

In [2]:
df = pd.read_csv('reference_sent.csv')
df.head()

,ID,overallLabel,Long,Lat,UTMzone,centxUTM,centyUTM,type,p1_date,p1,...,p69_date,p69,p70_date,p70,p71_date,p71,p72_date,p72,p73_date,p73
0,1,VLmaj,-52.379792,-23.297147,22,358905,-2577075,Natural browning,NaN,noObs,...,20220908.0,noChange,NaN,noChange,20220918.0,noChange,20220923.0,noChange,NaN,noChange
1,2,noChange,129.850328,-29.577240,52,582345,-3272235,No Change,20211004.0,noChange,...,20220910.0,noChange,20220914.0,noChange,20220920.0,noChange,20220924.0,noChange,NaN,noChange
2,3,VLmin,-98.600559,33.771004,14,536985,3736845,Natural browning,20211003.0,noChange,...,20220908.0,noChange,20220913.0,noChange,20220918.0,noChange,20220924.0,noChange,NaN,noChange
3,4,VLmaj,19.332894,4.574903,34,315045,505875,Fire,20211003.0,noChange,...,20220908.0,noChange,20220913.0,noChange,20220919.0,noChange,NaN,noChange,20220927.0,noChange
4,5,VLmaj,115.619006,63.108089,50,430305,6998385,Fire,20211001.0,noObs,...,NaN,noChange,NaN,noChange,20220918.0,noChange,20220924.0,noChange,NaN,noChange


In [3]:
geometry = [Point(xy) for xy in zip(df.Long, df.Lat)]
gdf = gpd.GeoDataFrame(df,geometry=geometry, crs= 4326)
gdf.head()

,ID,overallLabel,Long,Lat,UTMzone,centxUTM,centyUTM,type,p1_date,p1,...,p69,p70_date,p70,p71_date,p71,p72_date,p72,p73_date,p73,geometry
0,1,VLmaj,-52.379792,-23.297147,22,358905,-2577075,Natural browning,NaN,noObs,...,noChange,NaN,noChange,20220918.0,noChange,20220923.0,noChange,NaN,noChange,POINT (-52.37979 -23.29715)
1,2,noChange,129.850328,-29.577240,52,582345,-3272235,No Change,20211004.0,noChange,...,noChange,20220914.0,noChange,20220920.0,noChange,20220924.0,noChange,NaN,noChange,POINT (129.85033 -29.57724)
2,3,VLmin,-98.600559,33.771004,14,536985,3736845,Natural browning,20211003.0,noChange,...,noChange,20220913.0,noChange,20220918.0,noChange,20220924.0,noChange,NaN,noChange,POINT (-98.60056 33.77100)
3,4,VLmaj,19.332894,4.574903,34,315045,505875,Fire,20211003.0,noChange,...,noChange,20220913.0,noChange,20220919.0,noChange,NaN,noChange,20220927.0,noChange,POINT (19.33289 4.57490)
4,5,VLmaj,115.619006,63.108089,50,430305,6998385,Fire,20211001.0,noObs,...,noChange,NaN,noChange,20220918.0,noChange,20220924.0,noChange,NaN,noChange,POINT (115.61901 63.10809)


In [5]:
bucket_name = 'opera-calval-database-dist'
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.client('s3')
s3_resource = session.resource('s3')

In [6]:
newRow_bytes = bytes(gdf.to_json(drop_id=True).encode('UTF-8'))
s3object = s3_resource.Object(bucket_name, 'global_sample.geojson')
s3object.put(Body=newRow_bytes)

{'ResponseMetadata': {'RequestId': 'XCBBXX4JZCYTWFFR',
  'HostId': 'fff8nIfg4FPQockubMe8HwZjClnx46w7Bp5NWzCEZQTis/uk+MfYfze3Ep1cZHWB5euORShCngw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'fff8nIfg4FPQockubMe8HwZjClnx46w7Bp5NWzCEZQTis/uk+MfYfze3Ep1cZHWB5euORShCngw=',
   'x-amz-request-id': 'XCBBXX4JZCYTWFFR',
   'date': 'Mon, 20 Mar 2023 22:54:50 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"f776475b2b72548eefe35a2c8b4eb697"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"f776475b2b72548eefe35a2c8b4eb697"',
 'ServerSideEncryption': 'AES256'}